     No of Executors = No of Partitions = No of tasks

In [0]:
df = spark.read.csv("/Volumes/@azureadb/pyspark/split/split_to_new_columns.csv", header=True, inferSchema=True)
# display(df)
display(df.limit(10))

start_date,product_url,category,default_group,source_target,cloud_flatform,database_remarks,session_id,session_name,session_type,sessions,product_id,load_datetime
2025-08-25,shop.sony.bpl,mobile,wifi-network,amazon.it / referral,azure / aws / gcc,azure / aws / gcc,9876543,first_visit,Not Available,1,409516064,2025-09-02T19:10:35
2025-08-26,shop.sony.bpl,mobile,wifi-network,amazon.it / referral,azure / aws / gcc,azure / aws / gcc,9876544,purchase,Not Available,1,409516064,2025-09-02T19:10:36
2025-08-27,shop.sony.bpl,mobile,wifi-network,amazon.it / referral,azure / aws / gcc,azure / aws / gcc,9876545,search,Not Available,1,409516064,2025-09-02T19:10:37
2025-08-28,shop.sony.bpl,mobile,wifi-network,(data not available),azure / aws / gcc,amazon.it / referral,9876546,search,Not Available,1,409516064,2025-09-02T19:10:38
2025-08-29,shop.sony.bpl,mobile,wifi-network,sms / socialreply,azure / aws / gcc,azure / aws / gcc,9876547,search,Not Available,1,409516064,2025-09-02T19:10:39
2025-08-30,shop.sony.bpl,mobile,wifi-network,(not set),azure / aws / gcc,azure / aws / gcc,9876548,add_to_cart,Not Available,1,409516064,2025-09-02T19:10:40
2025-08-31,shop.sony.bpl,mobile,wifi-network,(not set),azure / aws / gcc,sms / socialreply,9876549,add_to_cart,Not Available,1,409516064,2025-09-02T19:10:41
2025-09-01,shop.sony.bpl,mobile,wifi-network,(none) / (direct),azure / aws / gcc,azure / aws / gcc,9876550,add_to_cart,Not Available,1,409516064,2025-09-02T19:10:42
2025-09-02,shop.sony.bpl,mobile,wifi-network,flipkart / referral,azure / aws / gcc,(data not available),9876551,add_to_cart,Not Available,1,409516064,2025-09-02T19:10:43
2025-09-03,shop.sony.bpl,mobile,wifi-network,(data not available),azure / aws / gcc,azure / aws / gcc,9876552,add_to_cart,Not Available,1,409516064,2025-09-02T19:10:44


In [0]:
num_partitions = df.rdd.getNumPartitions()
print(num_partitions)

In [0]:
from pyspark.sql import functions as F

df1 = df.selectExpr("count(start_date) as total_count") \
        .withColumn("partition_id", F.spark_partition_id())

df1.explain(True)
display(df1)

== Parsed Logical Plan ==
Project [total_count#13534L, SPARK_PARTITION_ID() AS partition_id#13537]
+- Aggregate [count(start_date#13383) AS total_count#13534L]
   +- Relation [start_date#13383,product_url#13384,category#13385,default_group#13386,source_target#13387,cloud_flatform#13388,database_remarks#13389,session_id#13390,session_name#13391,session_type#13392,sessions#13393,product_id#13394,load_datetime#13395] csv

== Analyzed Logical Plan ==
total_count: bigint, partition_id: int
Project [total_count#13534L, SPARK_PARTITION_ID() AS partition_id#13537]
+- Aggregate [count(start_date#13383) AS total_count#13534L]
   +- Relation [start_date#13383,product_url#13384,category#13385,default_group#13386,source_target#13387,cloud_flatform#13388,database_remarks#13389,session_id#13390,session_name#13391,session_type#13392,sessions#13393,product_id#13394,load_datetime#13395] csv

== Optimized Logical Plan ==
Aggregate [count(start_date#13383) AS total_count#13534L, SPARK_PARTITION_ID() AS pa

total_count,partition_id
100,0


In [0]:
df1.explain(False)

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- == Initial Plan ==
   ColumnarToRow
   +- PhotonResultStage
      +- PhotonAgg(keys=[], functions=[finalmerge_count(merge count#13425L) AS count(start_date)#13421L])
         +- PhotonShuffleExchangeSource
            +- PhotonShuffleMapStage ENSURE_REQUIREMENTS, [id=#8562]
               +- PhotonShuffleExchangeSink SinglePartition
                  +- PhotonAgg(keys=[], functions=[partial_count(start_date#13383) AS count#13425L])
                     +- PhotonRowToColumnar
                        +- FileScan csv [start_date#13383] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[dbfs:/Volumes/@azureadb/pyspark/split/split_to_new_columns.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<start_date:date>


== Photon Explanation ==
The query is fully supported by Photon.


In [0]:
from pyspark.sql import functions as F

df2 = df.repartition(4)
df2 = df2.withColumn("partition_id", F.spark_partition_id())
df2 = df2.groupBy("partition_id").agg(F.count("start_date").alias("total_count"))

df2.explain(False)
display(df2)

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- == Initial Plan ==
   ColumnarToRow
   +- PhotonResultStage
      +- PhotonGroupingAgg(keys=[partition_id#13481], functions=[finalmerge_count(merge count#13485L) AS count(start_date)#13483L])
         +- PhotonShuffleExchangeSource
            +- PhotonShuffleMapStage ENSURE_REQUIREMENTS, [id=#8919]
               +- PhotonShuffleExchangeSink hashpartitioning(partition_id#13481, 16)
                  +- PhotonGroupingAgg(keys=[partition_id#13481], functions=[partial_count(start_date#13383) AS count#13485L])
                     +- PhotonProject [start_date#13383, SPARK_PARTITION_ID() AS partition_id#13481]
                        +- PhotonShuffleExchangeSource
                           +- PhotonShuffleMapStage REPARTITION_BY_NUM, [id=#8911]
                              +- PhotonShuffleExchangeSink RoundRobinPartitioning(4)
                                 +- PhotonSort [start_date#13383 ASC NULLS FIRST]
                      

partition_id,total_count
0,25
1,25
2,25
3,25
